# Exercise 3.2

Import the pySpark libraries

In [1]:
import sys
!conda install --yes --prefix {sys.prefix} -c conda-forge pyspark

Solving environment: done

# All requested packages already installed.



In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, size

Connect to a local Spark session

In [3]:
spark = SparkSession.builder.appName("Packt").getOrCreate()

NotADirectoryError: [Errno 20] Not a directory: '/home/bas/Programs/spark-2.4.4-bin-hadoop2.7/bin/pyspark/./bin/spark-submit'

Read the raw data from a CSV file

In [ ]:
data = spark.read.csv('../../Datasets/netflix_titles_nov_2019.csv', header='true')

Take only the movies

In [ ]:
movies = data.filter((col('type') == 'Movie') & (col('release_year') == 2019))

Add a column with the number of actors

In [ ]:
transformed = movies.withColumn('count_cast', size(split(movies['cast'], ',')))

Select a subset of columns to store

In [ ]:
selected = transformed.select('title', 'director', 'count_cast', 'cast', 'rating', 'release_year', 'type')

Write the contents of the DataFrame to disk

In [ ]:
selected.write.csv('transformed.csv', header='true')